1. Liste todas as cidades e os países aos quais pertencem.
2. Liste todas as cidades que são capitais.
3. Liste todos os atributos dos países onde a expectativa de vida é menor que 70 anos.
4. Liste todas as capitais e as populações dos países cujos PIB é maior que 1 trilhão de dólares.
5. Quais é o nome e a população da capital do país onde o rio St. Lawrence tem sua nascente.
6. Qual é a média da população das cidades que não são capitais.
7. Para cada continente retorne o PIB médio de seus países.
8. Para cada país onde pelo menos 2 rios tem nascente, encontre o comprimento do menor rio.
9. Liste os países cujo PIB é maior que o PIB do Canada.

In [1]:
import sqlite3

In [2]:
pais_table =    '''CREATE TABLE IF NOT EXISTS Pais(
                Nome varchar(35) PRIMARY KEY,
                Continente varchar(35), 
                Pop real, 
                PIB real, 
                Expec_vida real
                )
                '''
cidade_table =  '''CREATE TABLE IF NOT EXISTS Cidade(
                Nome varchar(35) PRIMARY KEY,
                Pais varchar(35) REFERENCES Pais (Nome),
                Pop real, 
                Capital varchar(1))
                '''
rio_table =     '''CREATE TABLE IF NOT EXISTS Rio(
                Nome varchar(35),
                Nascente TEXT,
                Pais TEXT, 
                Comprimento REAL
                )'''


In [3]:
engine = sqlite3.connect('sql.db')

In [4]:
cursor = engine.cursor()

cursor.execute("PRAGMA foreign_keys = ON").fetchall()

[]

In [5]:
cursor.execute(pais_table)

In [6]:
cursor.execute(cidade_table)

In [7]:
cursor.execute(rio_table)

In [8]:
values_pais = [('Canada', 'Am. Norte', 30.1, 658, 77.08),('Mexico', 'Am. Norte', 107.5,694, 69.1),('Brasil', 'Am. Sul', 183.3, 10004, 65.2),('USA', 'Am. Norte', 270.0, 8003, 75.5)]

In [9]:
values_cidades = [('Washington', 'USA', 3.3, 'S'), ('Monterrey','Mexico',2.0, 'N'), ('Brasilia', 'Brasil', 1.5, 'S'), ('São Paulo', 'Brasil', 15.0, 'N'), ('Ottawa', 'Canada', 0.8, 'S'), ('Cid. Mexico', 'Mexico', 14.1, 'S')]

In [10]:
rio_values = [('St.Lawrence', 'USA', 'USA', 3.3), ('Grande', 'USA', 'Mexico', 2.0), ('Parana', 'Brasil', 'Brasil', 1.5), ('Mississipi', 'USA', 'USA', 15.0)]

In [11]:
cursor.executemany('INSERT INTO Pais VALUES(?,?,?,?,?)', values_pais)

In [12]:
cursor.execute('SELECT * FROM Pais').fetchall()

[('Canada', 'Am. Norte', 30.1, 658.0, 77.08),
 ('Mexico', 'Am. Norte', 107.5, 694.0, 69.1),
 ('Brasil', 'Am. Sul', 183.3, 10004.0, 65.2),
 ('USA', 'Am. Norte', 270.0, 8003.0, 75.5)]

In [13]:
cursor.executemany('INSERT INTO Cidade VALUES(?,?,?,?)', values_cidades)

In [14]:
cursor.execute('SELECT * FROM Cidade').fetchall()

[('Washington', 'USA', 3.3, 'S'),
 ('Monterrey', 'Mexico', 2.0, 'N'),
 ('Brasilia', 'Brasil', 1.5, 'S'),
 ('São Paulo', 'Brasil', 15.0, 'N'),
 ('Ottawa', 'Canada', 0.8, 'S'),
 ('Cid. Mexico', 'Mexico', 14.1, 'S')]

In [15]:
cursor.executemany('INSERT INTO Rio VALUES(?,?,?,?)', rio_values)

In [16]:
cursor.execute('SELECT * FROM Rio').fetchall()

[('St.Lawrence', 'USA', 'USA', 3.3),
 ('Grande', 'USA', 'Mexico', 2.0),
 ('Parana', 'Brasil', 'Brasil', 1.5),
 ('Mississipi', 'USA', 'USA', 15.0)]

## 1. Liste todas as cidades e os países aos quais pertencem.

In [18]:
sql_str1 = '''SELECT Nome, Pais
FROM Cidade'''
cursor.execute(sql_str1).fetchall()

[('Washington', 'USA'),
 ('Monterrey', 'Mexico'),
 ('Brasilia', 'Brasil'),
 ('São Paulo', 'Brasil'),
 ('Ottawa', 'Canada'),
 ('Cid. Mexico', 'Mexico')]

## 2. Liste todas as cidades que são capitais.

In [19]:
sql_str2 = '''SELECT Nome
FROM Cidade
WHERE Capital = "S"
'''
cursor.execute(sql_str2).fetchall()

[('Washington',), ('Brasilia',), ('Ottawa',), ('Cid. Mexico',)]

## 3. Liste todos os atributos dos países onde a expectativa de vida é menor que 70 anos.

In [20]:
sql_str3 = '''SELECT *
FROM Pais
WHERE Expec_vida < 70
'''
cursor.execute(sql_str3).fetchall()

[('Mexico', 'Am. Norte', 107.5, 694.0, 69.1),
 ('Brasil', 'Am. Sul', 183.3, 10004.0, 65.2)]

## 4. Liste todas as capitais e as populações dos países cujos PIB é maior que 1 trilhão de dólares.

In [22]:
sql_str4 = '''SELECT Cidade.Nome, Pais.Pop
FROM Pais
LEFT JOIN Cidade
ON Pais.Nome = Cidade.Pais
WHERE Pais.Pib > 1000
AND Cidade.Capital = 'S'
'''
cursor.execute(sql_str4).fetchall()

[('Washington', 270.0), ('Brasilia', 183.3)]

## 5. Qual é o nome e a população da capital do país onde o rio St. Lawrence tem sua nascente.

In [24]:
sql_str5 =  '''SELECT Cidade.Nome, Cidade.Pop
            FROM Cidade
            LEFT JOIN Rio
            ON Cidade.Pais = Rio.Nascente
            WHERE Rio.Nome = 'St.Lawrence'
            '''
cursor.execute(sql_str5).fetchall()

[('Washington', 3.3)]

## 6. Qual é a média da população das cidades que não são capitais.

In [26]:
sql_str6 =  '''SELECT avg(Cidade.Pop)
            FROM Cidade
            WHERE Cidade.Capital = "N"
            '''
cursor.execute(sql_str6).fetchall()

[(8.5,)]

## 7. Para cada continente retorne o PIB médio de seus países.

In [27]:
sql_str7 =  '''SELECT avg(Pais.Pib)
            FROM Pais
            GROUP BY Continente
            '''
cursor.execute(sql_str7).fetchall()

[(3118.3333333333335,), (10004.0,)]

## 8. Para cada país onde pelo menos 2 rios tem nascente, encontre o comprimento do menor rio.

In [73]:
sql_str8 =  '''WITH nascente_rio AS (SELECT Nascente as Nascente, count(*) as Contagem
            FROM Rio
            GROUP BY Nascente
            HAVING count(*) > 1
            )

            SELECT Rio.Nome, min(Rio.Comprimento)
            FROM Rio, nascente_rio
            WHERE nascente_rio.Nascente = Rio.Nascente
            '''
cursor.execute(sql_str8).fetchall()

[('Grande', 2.0)]

## 9. Liste os países cujo PIB é maior que o PIB do Canada.

In [49]:
sql_str9 =  '''WITH busca_pib AS (SELECT Pais.Nome as nome, Pais.PIB as canada_pib
            FROM Pais
            WHERE Pais.Nome like "Canada")
            
            SELECT Pais.Nome, Pais.PIB
            FROM Pais, busca_pib
            WHERE Pais.PIB > busca_pib.canada_pib
            '''
cursor.execute(sql_str9).fetchall()

[('Mexico', 694.0), ('Brasil', 10004.0), ('USA', 8003.0)]